In [ ]:
# Reference
# @Author gachonyws / face-mask-detector
# @Link : https://github.com/gachonyws/face-mask-detector

In [9]:
from tensorflow.keras.layers import Input, AveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

import numpy as np


In [3]:
with_mask_paths = ["./data/with_mask/with_mask_" + str(i) + ".jpg" for i in range(1, 3725 + 1)]
without_mask_paths = ["./data/without_mask/without_mask_" + str(i)  + ".jpg" for i in range(1, 3828 + 1)]

data = []
label = []

def load_data(path_arr, target_arr, target_size, _label):
    for path in path_arr:
        img = load_img(path, target_size = target_size)
        img = img_to_array(img)
        img = preprocess_input(img)
        target_arr.append(img)
        label.append(_label)

load_data(with_mask_paths, data, (224, 224, 3), 1)
load_data(without_mask_paths, data, (224, 224, 3), 0)


c:\users\juhyeon park\anaconda3\envs\test_gpu\lib\site-packages\PIL\Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [4]:
data = np.array(data, dtype = "float32")
label = np.array(label)

_label_binarizer = LabelBinarizer()

label = _label_binarizer.fit_transform(label)
label = to_categorical(label)

In [5]:
(train_x, test_x, train_y, test_y) = train_test_split(data, label, test_size=0.15, stratify=label, random_state=42)

In [6]:
base_model = MobileNetV2(weights = "imagenet", include_top = False,
                          input_tensor = Input(shape=(224,224,3)))

head_model = base_model.output
head_model = AveragePooling2D(pool_size = (7,7))(head_model)
head_model = Flatten()(head_model)
head_model = Dense(128, activation='relu')(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(2, activation="softmax")(head_model)

CNN = Model(inputs = base_model.input, outputs = head_model)


c:\users\juhyeon park\anaconda3\envs\test_gpu\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [7]:
epochs = 10
batch_size = 100
optimizer = Adam(learning_rate=1e-4, decay = 1e-4 / epochs)
CNN.compile(loss="binary_crossentropy", optimizer = optimizer, metrics=['accuracy'])

history = CNN.fit(train_x, train_y, batch_size = batch_size, validation_data=(test_x, test_y),
                  epochs = epochs)

CNN.save("./checkpoint")


Train on 6420 samples, validate on 1133 samples
Epoch 1/10
6420/6420 [==============================] - 37s 6ms/sample - loss: 0.1364 - accuracy: 0.9430 - val_loss: 0.0266 - val_accuracy: 0.9903

In [10]:
# Model Load Test
test_model = load_model("./checkpoint")


In [13]:
test_img = []
test_img_path = ["./testset/" +  str(i) + ".jpg" for i in range(1, 3)]
for path in test_img_path:
    img = load_img(path, target_size=(224, 224, 3))
    img = img_to_array(img)
    img = preprocess_input(img)
    test_img.append(img)
test_img = np.array(test_img, np.float32)

results = test_model.predict(test_img)

# 0 : nomask, 1 : mask
for result in results:
    print(np.argmax(result))

0
1
